<a href="https://colab.research.google.com/github/sonupatel24/air-quality-forecasting/blob/main/Streamlit_Air_Quality_Forecasting_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import pandas as pd
import numpy as np
from prophet import Prophet

In [3]:
df = pd.read_csv('/content/air_quality_clean.csv')

In [1]:
pip install --upgrade prophet

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/air_quality_clean.csv')
df['Date'] = pd.to_datetime(df['Date'])
monthly_df = df.set_index('Date').resample('M')['Air_Quality'].mean().reset_index()
monthly_df.columns = ['ds', 'y']
monthly_df['y'] = np.log(monthly_df['y'])
print("Monthly aggregated and log-transformed data:")
print(monthly_df.head())
print(monthly_df.info())

Monthly aggregated and log-transformed data:
          ds         y
0 2018-07-31  3.866614
1 2018-08-31  3.758992
2 2018-09-30  4.210954
3 2018-10-31  4.914273
4 2018-11-30  4.780577
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      66 non-null     datetime64[ns]
 1   y       66 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.2 KB
None


/tmp/ipython-input-3308954157.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_df = df.set_index('Date').resample('M')['Air_Quality'].mean().reset_index()


In [7]:
from prophet import Prophet

m = Prophet(yearly_seasonality=True, daily_seasonality=False)
m.fit(monthly_df)
print("Prophet model initialized and fitted successfully.")

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


Prophet model initialized and fitted successfully.


In [8]:
future = m.make_future_dataframe(periods=12, freq='M')
print("Future DataFrame created:")
print(future.tail())

Future DataFrame created:
           ds
73 2024-08-31
74 2024-09-30
75 2024-10-31
76 2024-11-30
77 2024-12-31


/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [9]:
forecast = m.predict(future)
print("Forecast generated:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

Forecast generated:
          ds      yhat  yhat_lower  yhat_upper
0 2018-07-31  3.990087    3.800896    4.172866
1 2018-08-31  4.105895    3.911455    4.296863
2 2018-09-30  4.158455    3.977776    4.364457
3 2018-10-31  4.657229    4.468458    4.836986
4 2018-11-30  4.756461    4.550168    4.939003


In [10]:
forecast['yhat'] = np.exp(forecast['yhat'])
forecast['yhat_lower'] = np.exp(forecast['yhat_lower'])
forecast['yhat_upper'] = np.exp(forecast['yhat_upper'])

monthly_df['y'] = np.exp(monthly_df['y'])

print("Forecast inverse transformed:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

Forecast inverse transformed:
          ds        yhat  yhat_lower  yhat_upper
0 2018-07-31   54.059608   44.741276   64.901197
1 2018-08-31   60.697072   49.971605   73.468958
2 2018-09-30   63.972600   53.398148   78.606718
3 2018-10-31  105.343752   87.222115  126.088741
4 2018-11-30  116.333445   94.648276  139.630930


In [11]:
import plotly.graph_objects as go

fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(
    x=monthly_df['ds'],
    y=monthly_df['y'],
    mode='lines',
    name='Historical Air Quality',
    line=dict(color='blue')
))

# Add forecasted data
fig.add_trace(go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat'],
    mode='lines',
    name='Forecasted Air Quality',
    line=dict(color='red')
))

# Add upper prediction interval
fig.add_trace(go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat_upper'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

# Add lower prediction interval and fill between upper and lower
fig.add_trace(go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat_lower'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.2)',
    name='Prediction Interval',
    line=dict(width=0)
))

# Update layout for better readability
fig.update_layout(
    title='Air Quality Forecast',
    xaxis_title='Date',
    yaxis_title='Air Quality (µg/m³)',
    hovermode='x unified'
)

fig.show()